# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [ ]:
import pandas as pd

In [ ]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**. 

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method. 

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [ ]:
# orders
orders.duplicated().sum()

0

In [ ]:
# orderlines
orderlines.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [ ]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [ ]:
print(f"5 missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [ ]:
orders = orders.loc[~orders.total_paid.isna(), :]

Should you have a significant number of missing values in the future, you have a choice: 
+ you can impute the values
+ you can take the values from other DataFrames, if they are present there
+ you can delete the values
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [ ]:
orders["created_date"] = pd.to_datetime(orders["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [ ]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

#### 4.1.2.&nbsp;`unit_price`

In [ ]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

ValueError: ignored

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have 2 decimal points. Let's see if there are any other numbers like this.

In [ ]:
orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [ ]:
two_dot_percentage = ((orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [ ]:
two_dot_order_ids_list = orderlines.loc[orderlines.unit_price.astype(str).str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines = orderlines.loc[~orderlines.id_order.isin(two_dot_order_ids_list)]

In [ ]:
orderlines.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [ ]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [ ]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Look for Duplicates

In [ ]:
products.duplicated().sum()

8746

In [ ]:
products.drop_duplicates(inplace=True)
print(products)


           sku                                           name  \
0      RAI0007              Silver Rain Design mStand Support   
1      APP0023              Apple Mac Keyboard Keypad Spanish   
2      APP0025               Mighty Mouse Apple Mouse for Mac   
3      APP0072  Apple Dock to USB Cable iPhone and iPod white   
4      KIN0007    Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM   
...        ...                                            ...   
19321  BEL0376        Belkin Travel Support Apple Watch Black   
19322  THU0060   Enroute Thule 14L Backpack MacBook 13 "Black   
19323  THU0061    Enroute Thule 14L Backpack MacBook 13 "Blue   
19324  THU0062     Enroute Thule 14L Backpack MacBook 13 "Red   
19325  THU0063   Enroute Thule 14L Backpack MacBook 13 "Green   

                                                    desc  price promo_price  \
0           Aluminum support compatible with all MacBook  59.99     499.899   
1              USB ultrathin keyboard Apple Mac Spanish.     

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


In [ ]:
products.drop_duplicates(inplace = True)
products[products['sku'].duplicated()]
 # manually extract the sku APP1197#duplicate overall is 8746 and sku is 8747 and one is blank that is why we also delete it
products[products["sku"]=='APP1197']
products.sku.drop_duplicates(keep='first')

0        RAI0007
1        APP0023
2        APP0025
3        APP0072
4        KIN0007
          ...   
19321    BEL0376
19322    THU0060
19323    THU0061
19324    THU0062
19325    THU0063
Name: sku, Length: 10522, dtype: object

In [ ]:
products_sku=products[products['sku']=='APP1197']
products_sku.drop_duplicates()


,sku,name,desc,price,promo_price,in_stock,type
7992,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282
8000,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,NaN,1305.59,0,1282


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10523 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10523 non-null  object
 1   name         10523 non-null  object
 2   desc         10523 non-null  object
 3   price        10477 non-null  object
 4   promo_price  10523 non-null  object
 5   in_stock     10523 non-null  int64 
 6   type         10523 non-null  object
dtypes: int64(1), object(6)
memory usage: 657.7+ KB


### Look for Missing values


In [ ]:
print(f"missing values represents {((products.type.isna().sum() / products.shape[0])*100).round(5)}% of the rows in our DataFrame")

missing values represents 0.47259% of the rows in our DataFrame


In [ ]:
products.type.isna().sum()

0

In [ ]:
print(f"missing values represents {((products.price.isna().sum() / products.shape[0])*100).round(5)}% of the rows in our DataFrame")

missing values represents 0.43478% of the rows in our DataFrame


In [ ]:
print(f"missing values represents {((products.desc.isna().sum() / products.shape[0])*100).round(6)}% of the rows in our DataFrame")

missing values represents 0.066163% of the rows in our DataFrame


In [ ]:
products = products.loc[~products.price.isna(), :]


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10104 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10104 non-null  object 
 1   name         10104 non-null  object 
 2   desc         10104 non-null  object 
 3   price        10104 non-null  float64
 4   promo_price  10104 non-null  object 
 5   in_stock     10104 non-null  int64  
 6   type         10104 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 631.5+ KB


### Check / Change Data types

In [ ]:
import pandas as pd
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10477 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10477 non-null  object
 1   name         10477 non-null  object
 2   desc         10477 non-null  object
 3   price        10477 non-null  object
 4   promo_price  10477 non-null  object
 5   in_stock     10477 non-null  int64 
 6   type         10477 non-null  object
dtypes: int64(1), object(6)
memory usage: 654.8+ KB


In [ ]:
products["price"] = pd.to_numeric(products["price"])

ValueError: ignored

In [ ]:
products.price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()

False    10104
True       373
Name: price, dtype: int64

In [ ]:
price_problems_number = products.loc[(products.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [ ]:
products.drop(columns=["promo_price"])

,sku,name,desc,price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,1,1364
...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,0,1392


In [ ]:
two_dot_percentage = ((products.price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()[1] / products.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 3.56% of the rows in our DataFrame


In [ ]:
two_dot_list = products.loc[products.price.astype(str).str.contains("\d+\.\d+\.\d+"), "sku"]
products = products.loc[~products.sku.isin(two_dot_list)]
products.shape[0]

10104

In [ ]:
products["price"] = pd.to_numeric(products["price"])


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10104 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10104 non-null  object 
 1   name         10104 non-null  object 
 2   desc         10104 non-null  object 
 3   price        10104 non-null  float64
 4   promo_price  10104 non-null  object 
 5   in_stock     10104 non-null  int64  
 6   type         10104 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 631.5+ KB


In [ ]:
products["promo_price"] = pd.to_numeric(products["promo_price"])

ValueError: ignored

In [ ]:
products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()

False    5866
True     4284
Name: promo_price, dtype: int64

In [ ]:
two_dot_percentage = ((products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+").value_counts()[1] / products.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 42.21% of the rows in our DataFrame


In [ ]:
two_dot_list = products.loc[products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"), "sku"]
products = products.loc[~products.sku.isin(two_dot_list)]
products.shape[0]

5866

In [ ]:
products["promo_price"] = pd.to_numeric(products["promo_price"])

In [ ]:
from google.colab import files
orders.to_csv("orders_mecl.csv",index=False)
files.download("orders_mecl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
orderlines.to_csv("orderlines_mecl.csv",index=False)
files.download("orderlines_mecl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
products.to_csv("products_mecl.csv",index=False)
files.download("products_mecl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>